In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt



In [19]:
import pandas as pd

# File path
file_path = "xi_plus_covariance.dat"

try:
    # Read the file using pandas
    # Adjust 'delim_whitespace=True' if the file is space-separated
    C_true = pd.read_csv(file_path, delim_whitespace=True, header=None)

    # Display the first few rows of the data
    #print(data.head())

except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

print(C_true.shape)
print(C_true[0].shape)


(15, 15)
(15,)


/var/folders/kw/m2f3pwyj6s90fdg5k6m6b4k80000gn/T/ipykernel_96783/3460718957.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  C_true = pd.read_csv(file_path, delim_whitespace=True, header=None)


In [31]:
psi_true = np.linalg.inv(C_true) #true precision matrix

N_measurement = 50
N_trial = 1000
## A_hat will be Psi_true @ C_hat
## B_hat will be Psi_sample @ C_true
traces_A = []
traces_B = []
dim = C_true.shape[0]
for trial in range(N_trial):
    data = np.random.multivariate_normal(mean = np.zeros(dim),cov = C_true, size = N_measurement)
    C_sample = np.cov(data, rowvar=False)

    psi_sample_A = np.linalg.inv(C_sample)
    A_hat = psi_true @C_sample
    traces_A.append(np.trace(A_hat))

    psi_sample_B = np.linalg.inv(C_sample)
    B_hat = psi_sample_B@C_true
    traces_B.append(np.trace(B_hat))
    

mean_trace_A = np.mean(traces_A)
mean_trace_B = np.mean(traces_B)

print(mean_trace_A)
print(mean_trace_B)

#As N_measurement increases, both means should approach dimension but from opposite sides.
#Trace(A_hat) is expected to be less than dim while Trace(B_hat) is expected to be greater than dim for finite N_measurement.

mean_trace_A = np.mean(traces_A)
err_trace_A = np.std(traces_A) / np.sqrt(N_trials)

print("\n--- Part C: Trace(A) = Trace(C_inv * C_hat) ---")
print(f"Expected Trace: {dim} (Identity matrix dimension)")
print(f"Measured Trace: {mean_trace_A:.4f} +/- {err_trace_A:.4f}")
print("Conclusion: C_hat is an UNBIASED estimator of C.")

# Part E Results [cite: 32-35]
mean_trace_B = np.mean(traces_B)
err_trace_B = np.std(traces_B) / np.sqrt(N_trials)

print("\n--- Part E: Trace(B) = Trace(C_hat_inv * C_true) ---")
print(f"Expected Trace (if unbiased): {dim}")
print(f"Measured Trace: {mean_trace_B:.4f} +/- {err_trace_B:.4f}")

bias_factor = (N_measurements - 1) / (N_measurements - dim - 2)
print(f"\nTheoretical Bias Factor (Hartlap): {bias_factor:.4f}")
print(f"Theoretical Prediction (15 * Factor): {dim * bias_factor:.4f}")

print("\nConclusion: C_hat_inv is a BIASED estimator of C_inv.")
    


14.995501313152596
22.261709941540637

--- Part C: Trace(A) = Trace(C_inv * C_hat) ---
Expected Trace: 15 (Identity matrix dimension)
Measured Trace: 14.9955 +/- 0.0244
Conclusion: C_hat is an UNBIASED estimator of C.

--- Part E: Trace(B) = Trace(C_hat_inv * C_true) ---
Expected Trace (if unbiased): 15
Measured Trace: 22.2617 +/- 0.0539

Theoretical Bias Factor (Hartlap): 1.0163
Theoretical Prediction (15 * Factor): 15.2442

Conclusion: C_hat_inv is a BIASED estimator of C_inv.


In [57]:
from scipy.stats import norm

sigma = 0.5
n_experiment = 10000
alpha_true_values = [0,1] 

def calculate_coverage(alpha_true, sigma = 0.5, n_sims = n_experiment):
    d_sims = np.random.normal(loc = alpha_true, scale = sigma, size = n_sims)
    alpha_grid = np.linspace(-3,3,n_sims)
    dx = alpha_grid[1] - alpha_grid[0]

    success_count = 0   
    for d in d_sims:
        log_likelihood = -0.5 * ((d - alpha_grid**3)/sigma)**2
        # Normalize (subtract max to prevent underflow)
        posterior = np.exp(log_likelihood - np.max(log_likelihood))
        norm = trapz(posterior, alpha_grid)
        posterior /=norm

        sorted_indices = np.argsort(posterior)[::-1]# this will give the indices that would sort the array in descending order
        sorted_posterior = posterior[sorted_indices]

        #cumulative probability

        cumulative_posterior = np.cumsum(sorted_posterior) * dx
        idx_cutoff = np.searchsorted(cumulative_posterior, 0.683)
        included_indices = sorted_indices[:idx_cutoff + 1]
        
        min_alpha_cred = np.min(alpha_grid[included_indices])
        max_alpha_cred = np.max(alpha_grid[included_indices])

        if min_alpha_cred <= alpha_true <= max_alpha_cred:
            success_count += 1

    return success_count / n_sims


coverage_0 = calculate_coverage(alpha_true = 0, sigma = sigma, n_sims = n_experiment)
coverage_1 = calculate_coverage(alpha_true = 1, sigma = sigma, n_sims = n_experiment)

print(f"Coverage Probability for alpha_true = 0: {coverage_0:.4f}")
print(f"Coverage Probability for alpha_true = 1: {coverage_1:.4f}")



Coverage Probability for alpha_true = 0: 0.7763
Coverage Probability for alpha_true = 1: 0.7075


In [45]:
import numpy as np
from scipy.integrate import trapz
def calculate_coverage(alpha_true, sigma=0.5, n_sims=2000):
    """
    Simulates the coverage of Bayesian credible intervals for the model d = alpha^3.
    """    # 
    d_sims = np.random.normal(loc=alpha_true**3, scale=sigma, size=n_sims)

    # 2. Define a grid for alpha to calculate the posterior
    # We need a range that covers the likely values of alpha given the noise.
    alpha_grid = np.linspace(-3.0, 3.0, 2000)
    dx = alpha_grid[1] - alpha_grid[0]
    
    hits = 0
    
    for d in d_sims:
        # 3. Calculate Posterior for each experiment
        # Likelihood P(d|alpha) = exp( -0.5 * (d - alpha^3)^2 / sigma^2 )
        log_likelihood = -0.5 * (d - alpha_grid**3)**2 / sigma**2
        
        # Normalize (subtract max to prevent underflow)
        posterior = np.exp(log_likelihood - np.max(log_likelihood))
        norm = trapz(posterior, alpha_grid)
        if norm == 0: continue
        posterior /= norm
        
        # 4. Find 68.3% Credible Interval (Highest Posterior Density)
        # Sort grid points by probability density (highest to lowest)
        sorted_indices = np.argsort(posterior)[::-1]
        sorted_post = posterior[sorted_indices]
        
        # Cumulative sum to find the top 68.3% mass
        cumulative_prob = np.cumsum(sorted_post * dx)
        
        # Find how many points we need to include to reach 68.3%
        # (np.searchsorted finds the index where condition is met)
        idx_cutoff = np.searchsorted(cumulative_prob, 0.683)
        included_indices = sorted_indices[:idx_cutoff+1]
        
        # 5. Check if true value is inside the credible region
        # We define the region by the min and max of the included alpha points
        min_alpha_cred = np.min(alpha_grid[included_indices])
        max_alpha_cred = np.max(alpha_grid[included_indices])
        
        if min_alpha_cred <= alpha_true <= max_alpha_cred:
            hits += 1
            
    return hits / n_sims

# --- Run the Simulations ---

# Case 1: True alpha is 0 (Highly non-linear region)
coverage_0 = calculate_coverage(alpha_true=0.0)
print(f"Coverage for alpha=0: {coverage_0:.3f}")
# Expected result: ~0.78 (Over-coverage)

# Case 2: True alpha is 1 (Locally linear region)
coverage_1 = calculate_coverage(alpha_true=1.0)
print(f"Coverage for alpha=1: {coverage_1:.3f}")
# Expected result: ~0.68 (Matches Frequentist expectation)

/var/folders/kw/m2f3pwyj6s90fdg5k6m6b4k80000gn/T/ipykernel_96783/1458568530.py:23: DeprecationWarning: 'scipy.integrate.trapz' is deprecated in favour of 'scipy.integrate.trapezoid' and will be removed in SciPy 1.14.0
  norm = trapz(posterior, alpha_grid)


Coverage for alpha=0: 0.775
Coverage for alpha=1: 0.694
